In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import opendatasets as od
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import collections
import random
import itertools
from helper import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
from sklearn.dummy import DummyClassifier


[nltk_data] Downloading package stopwords to /Users/abyte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/abyte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
bugs_df= pd.read_csv("bugs_calendar.csv")
# bugs_eclipse = pd.read_csv("bugs_eclipse.csv")
# bugs_firefox= pd.read_csv("bugs_firefox.csv")
# bugs_calendar= pd.read_csv("bugs_calendar.csv")

# bugs_eclipse['Type'] = np.where(bugs_eclipse['Severity'] == 'enhancement', "enhancement", "defect")
# bugs_df = pd.concat([bugs_firefox,bugs_calendar,bugs_eclipse])

# Dropped rows with severity level '--'
bugs_df = bugs_df[bugs_df["Severity"].str.contains("--")==False].reset_index()

#Dropped rows with Type "Enhancement" and "Task" because they are not a bug but a new feature
indexSevere = bugs_df[ (bugs_df['Type'] == 'enhancement') & (bugs_df['Type'] == 'enhancement') ].index
bugs_df.drop(indexSevere , inplace=True)

indexSevere = bugs_df[ (bugs_df['Type'] == 'task') & (bugs_df['Type'] == 'task') ].index
bugs_df.drop(indexSevere , inplace=True)


#Catagorise the severity level into a Severe and Non Severe to make it a binary problem
bugs_df.loc[bugs_df["Severity"] == "blocker", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "critical", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "major", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "S1", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "S2", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "S3", "Severity"] = 'NonSevere'
bugs_df.loc[bugs_df["Severity"] == "normal", "Severity"] = 'NonSevere'
bugs_df.loc[bugs_df["Severity"] == "minor", "Severity"] = 'NonSevere'
bugs_df.loc[bugs_df["Severity"] == "trivial", "Severity"] = 'NonSevere'
bugs_df.loc[bugs_df["Severity"] == "S4", "Severity"] = 'NonSevere'


In [3]:
for i in range(0,2):
    TEST_SIZE = 0.2
    
    training_data, testing_data = train_test_split(bugs_df, test_size=TEST_SIZE)
    training_data, validation_data = train_test_split(training_data, test_size=TEST_SIZE)

    print(f"No. of training data: {training_data.shape[0]}")
    print(f"No. of validation data: {validation_data.shape[0]}")
    print(f"No. of testing data: {testing_data.shape[0]}")

    trainingdataset = len(training_data)
    testingdataset = len(testing_data) 
    validationdataset = len(validation_data)

    training_data_df=training_data.reset_index()
    validation_data_df=validation_data.reset_index()
    testing_data_df=testing_data.reset_index()
    print("------interation------", i)
    
    dict_resp = outer_loop(TEST_SIZE,bugs_df,trainingdataset,testingdataset,validationdataset,training_data_df,validation_data_df,testing_data_df,validation_data,testing_data)
    print(dict_resp)
    print("*******************************Dictionary Ends*******************************")
    mlclassifierresp =  mlclassifier_outerloop(TEST_SIZE,bugs_df, trainingdataset,testingdataset,validationdataset,training_data_df,validation_data_df,testing_data_df,training_data)
    print(mlclassifierresp)
       
    print("*******************************One Iteration completed*******************************")

No. of training data: 5405
No. of validation data: 1352
No. of testing data: 1690
------interation------ 0
*******************************Dictionary Ends*******************************
0.1 0.21935483870967742
0.5 0.26785714285714285
1 0.2693409742120344
5 0.3225806451612903
10 0.34660421545667447
20 0.3586206896551724
50 0.35535307517084286
100 0.37583892617449666
best c 100.0
0.1 0.22508038585209003
0.5 0.2889518413597734
1 0.31007751937984496
5 0.36960985626283366
10 0.39207920792079204
20 0.38314176245210724
50 0.379182156133829
100 0.3663003663003663
best c 10.0
0.1 0.22508038585209003
0.5 0.2889518413597734
1 0.31007751937984496
5 0.36960985626283366
10 0.39207920792079204
20 0.38314176245210724
50 0.379182156133829
100 0.3663003663003663
best c 10.0
    features      C               Model            confusionmatrix  Accuracy  \
0       1000  100.0                 SVM   [[1242, 141], [215, 92]]  0.789349   
1       1000    NaN       MultinomialNB  [[1208, 175], [190, 117]]  0.7840